![](https://raw.githubusercontent.com/stack-of-tasks/pinocchio/master/doc/images/pinocchio-logo-large.png)

# Pinocchio: rigib-body algorithms


In [ ]:
import magic_donotload

## Set up
We will need Pinocchio, the robot models stored in the package `example-robot-data`, a viewer (either GepettoViewer or MeshCat), some basic linear-algebra operators and the SciPy optimizers.

In [ ]:
import pinocchio as pin
import example_robot_data as robex
import numpy as np
from numpy.linalg import inv, pinv, eig, norm, svd, det
from scipy.optimize import fmin_bfgs
import time
import copy

## 1. Load and display the robot

Pinocchio is a library to compute different quantities related to the robot model, like body positions, inertias, gravity or dynamic effects, joint jacobians, etc. For that, we need first to define the robot model. The easiest solution is to load it from a [URDF model](https://ocw.tudelft.nl/course-lectures/2-2-1-introduction-to-urdf/). This can be done with the function `buildModelFromUrdf`.

The package `example-robot-data` proposes to directly load several URDF models with a single line of code. This is what we are going to use for the tutorial. Look at the code inside the package if you want to adapt it to the URDF model of your favorite robot.


In [ ]:
robot = robex.loadTalosArm()  # Load a 6-dof manipulator arm

The robot can be visualized in a viewer. Two viewers are proposed in this tutorial: Gepetto-Viewer and MeshCat. Both can display the robot model in an external windows managed by another processus.
- MeshCat is browser based, which makes it very convenient in the jupyter context, as it can also be embedded inside the notebook.
- Gepetto-Viewer is the historical viewer for Gepetto, and is more powerful. But you need to first run the command `gepetto-gui` in a terminal, before starting your python command.

In [ ]:
#Viewer = pin.visualize.GepettoVisualizer
Viewer = pin.visualize.MeshcatVisualizer

viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)
viz.display(robot.q0)

If chosing MeshCat, you can open the display directly inside the notebook.

In [ ]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

In both viewers, additional simple geometries can be added to enhance the visualization, although the syntax of the two viewers is not unified. For this tutorial, we provide a simple wrap-up for 3 methods to display a sphere, a box and place them in the scene. Check out the `vizutils.py` script if you are curious of the syntax, and would like to add more fancy primitives to your scene.

In [ ]:
import vizutils
vizutils.addViewerBox(viz, 'world/box', .05, .1, .2, [1., .2, .2, .5])
vizutils.addViewerSphere(viz,'world/ball', .05, [.2, .2, 1., .5])
vizutils.applyViewerConfiguration(viz, 'world/box', [0.5, -.2, .2, 1, 0, 0, 0])
vizutils.applyViewerConfiguration(viz, 'world/ball', [0.5, .2, .2, 1, 0, 0, 0])

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>robex.loadXXX</code> loads the model of the robot XXX
        <li><code>vizutils.applyViewerConfiguration(name, xyzquat_placement)</code> change the placement of the geometry named <code>name</code> in the viewer.
     </ul>

</div>

## 2. Pinocchio's philosophy (model, data and algorithms)

### Model vs data

Pinocchio is not extensively using the object-oriented design pattern. We rather regroup the informations and buffers in some important data structure, then access to these structures using algorithms written in static functions.

The two main data structures are `robot.model` and `robot.data`.

In [ ]:
rmodel = robot.model
rdata = rmodel.createData()

The `robot.model` structure contains all the *constant* information that the algorithms need to process. It is typically loaded from a file describing the model (URDF). The `robot.model` is typically constant and is not modified by algorithms.

The `robot.data` structure contains the memory buffers that the algorithms needs to store intermediary values or the final results to return to the user. 

### Joints names and indexes

You can get the list of the joint names with the following:

In [ ]:
for n in rmodel.names:
    print(n)

In what follows, we will specifically use the joint named `gripper_left_joint` (nothing specific, you can change it if you like). Its index can be obtained with:

In [ ]:
jointIndex = rmodel.getJointId("gripper_left_joint")

### A first algorithm: random configuration

Let's take a first example of algorithm. You can pick a random configuration by calling the algorithm `randomConfiguration`. This algorithm just needs the `robot.model` (as no intermediary buffer are needed).

In [ ]:
q = pin.randomConfiguration(rmodel)

### A second algorithm: forward kinematics

Another example is the algorithm to compute the foward kinematics. It recursively computes the placement of all the joint frames of the kinematic tree, and stores the results in `robot.data.oMj`, which is an array indexed by the joint indexes.

In [ ]:
pin.forwardKinematics(rmodel, rdata, q)
rdata.oMi[jointIndex]

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>robot.model</code> and <code>robot.data</code> structures
        <li><code>pin.randomConfiguration(robot.model)</code> to sample a random configuration
        <li><code>pin.forwardKinematics(robot.model, robot.data, q)</code> to compute joint frame placements.
        <li><code>robot.data.oMi[jointIdex]</code> to access a particular joint frame.
     </ul>

</div>

## 3. 3d cost: optimizing the end effector position

We will now define a first cost function, that penalizes the distance between the robot gripper and a target.
For that, let's define a target position `ptarget` of dimension 3. 

In [ ]:
ptarget = np.array([.5, .1, .3])

### Joints and frames
As explained upper, the position of the joint `jointIndex` is stored in `robot.data.oMi[jointIndex].translation`, and is recomputed by `pin.forwardKinematics`. 

In Pinocchio, each joint is defined by its joint frame, whose **placement** (*i.e.* position & orientation) is stored in `robot.data.oMi`. 

We also defined in addition other *operational* frames. They are defined by a fixed placement with respect to their parent joint frame. Denoting by `oMi` the placement of the parent joint frame (function of `q`), and by `iMf` the fixed placement of the operational frame with respect to the parent joint frame, the placement of the operational frame with respect to the world is easily computed by `oMf(q) = oMi(q) * iMf`. 

A complete list of available frames is stored in `robot.frames`. 


In [ ]:
for f in robot.model.frames:
    print(f.name)

All frame placements are computed directly by calling `pin.framesForwardKinematics` (or alternatively by calling `pin.updateFramePlacement` or `pin.updateFramePlacements` after a call to `pin.forwardKinematics`).

In [ ]:
pin.framesForwardKinematics(rmodel, rdata, q)

For the tutorial, we will use the frame `gripper_left_fingertip_1_link`:

In [ ]:
frameIndex = rmodel.getFrameId('gripper_left_fingertip_1_link')

### Cost-function template
In this tutorial, the cost functions will all be defined following the same template:

In [ ]:
class Cost:
    def __init__(self, rmodel, rdata, viz=None):  # add any other arguments you like
        self.rmodel = rmodel
        self.rdata  = rdata
        self.viz = viz
        
    def calc(self, q):
        ### Add the code to recompute your cost here
        cost = 0
        return cost
    
    def callback(self, q):
        if viz is None: 
            return
        # Display something in viz ...

We will see later that the callback can be used to display or print data during the optimization. For example, we may want to display the robot configuration, and visualize the target position with a simple sphere added to the viewer.

![Question](question.png) 
Implement a `Cost` class computing the quadratic cost `p(q) - ptarget`.

In [ ]:
%do_not_load -r 11-36 costs.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li> Frame placements are computed by <code>pin.framesForwardKinematics(rmodel, rdata, q)</code>, and accessed by <code>rdata.oMf[frameIndex]</code>.
        <li> Frame translation is a 3d array stored in <code>rdata.oMf[frameIndex].translation</code>.            </ul>

</div>

## 4. 6d cost: optimizing the end effector placement (position+orientation)

We will now define a cost function that penalizes the placement, i.e. rotation+translation between an operational frame and a target frame.

### Spatial algebra in Pinocchio
Most of the physical quantifies stored in Pinocchio are 6D quantities, that are elements of the so-called *Spatial Algebra*, following Roy Featherstone definition and namings. Featherstone's work, and in particular [his excellent 2008 book](https://link.springer.com/content/pdf/10.1007%2F978-1-4899-7560-7.pdf) is the basis for all the algorithms in Pinocchio.

Frame placement, formally elements of the Lie group SE(3), are represented in Pinocchio by an object of class `pin.SE3` containing a 3x3 rotation array and 3 translation vector. Placements can by multiplied and inverted.

In [ ]:
M1 = pin.SE3.Identity()
M2 = pin.SE3.Random()
print(M2, M2.rotation, M2.translation)
M3 = M1 * M2
M4 = M2.inverse()

SE(3) comes with `log` and `exp` operations. The `log` maps a SE3 placement into the 6D velocity that should be applied during 1 second to obtain this placement, starting from identity. In Pinocchio, 6D velocity are represented as `pin.Motion` object, and can be mapped to 6D array.

In [ ]:
nu = pin.log(M1)
print(nu.vector)

We will not need much of spatial algebra in this tutorial. See the class `pin.Motion`, `pin.Force`, `pin.Inertia` for more details. 

### Distances between frames
The `log` operator can be used to define a distance between placements. The norm of the log is evidently positive and null only if the placement is identity. Consequently, `log(M1.inverse() * M2)` is a positive scalar that is null only if `M1 == M2`.

![Question](question.png)
Following the same previous pattern, define a Cost6d class penalizing the distance between a frame attached to the robot and a reference fixed frame.

In [ ]:
%do_not_load -r 47-72 costs.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li>SE3 logarithm is implemented by <code>pin.log</code>, returns a <code>pin.Motion</code> class that can be converted into a vector with <code>pin.log(M).vector</code>.
     </ul>

</div>

## 5. Redundancy and introduction of a posture cost

We will now run the optimizer with the two cost functions defined above, and add a posture cost to regularize the rank-deficient hessian.

### Running the optimizer
We will use the optimizer BFGS from SciPy. It is quasi-newton algorithm, which means that it only needs the first-order derivatives (while having super-linear convergence). Even better, it automatically approximates the derivatives by finite differences if you don't provide them. 


In [ ]:
viz.viewer.jupyter_cell()

In [ ]:
cost = Cost3d(rmodel, rdata, viz=viz)
qguess = robot.q0.copy()
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)
viz.display(qopt)

### Redundancy

The arm of the robot Talos, that we are useing by default in the notebook, as 6 degrees of freedom (plus one for the gripper). If using the 3D cost function, there is a continuum of solutions, as the kinematic is redundant for achieve a pointing tasks. You can obtain different optimum by changing the initial guess. Each new run with a random initial guess gives you a new optimum.

In [ ]:
qguess = pin.randomConfiguration(rmodel)
qopt = fmin_bfgs(cost.calc, qguess, callback=cost.callback)
viz.display(qopt)

We will now add a small regularization to the cost, by optimizing a full-rank term on the robot posture, to make the solution unique indenpendantly from the robot kinematics and the considered task cost.

![Question](question.png)
Introduce a new cost function for penalizing the distance of the robot posture to a reference posture, for example `robot.q0`.

In [ ]:
%do_not_load -r 82-95 costs.py

### Optimize a sum of cost

Now we can define an ad-hoc cost that makes a sum of both costs.

In [ ]:
class SumOfCost:
    def __init__(self, costs, weights):
        self.costs = costs
        self.weights = np.array(weights)
    def calc(self, q):
        return sum(self.weights * [cost.calc(q) for cost in self.costs])
    
mycost = SumOfCost([Cost3d(rmodel, rdata), CostPosture(rmodel, rdata)], [1, 1e-3])

And we optimize this new cost.

In [ ]:
fmin_bfgs(mycost.calc, qguess)

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li>The BFGS solver is called by <code>fmin_bfgs(cost.calc, qguess, cost.callback)</code>.
     </ul>

</div>

## 6. Gravity cost: introducing the dynamic model

The posture cost is a nice regularization but requires a reference posture that may be hard to select. We will now define a cost to minimize the gravity torque, as an alternative regularization.

### Dynamics in Pinocchio

The whole-body dynamics equation can be written as:
$$M(q) a_q + b(q,v_q) = \tau_q   $$
where $q,v_q,a_q$ are the position, velocity and acceleration in the configuration space, $M(q)$ is the configuration-space inertia matrix, of size `robot.model.nv`x`robot.model.nv`, $b(q,v_q)$ gathers all the drift terms (Coriolis, centrifugal, gravity) and $\tau_q$ are the joint torques. We write $v_q$ and $a_q$ because in general $q$ (of size `robot.model.nq`) does not have the same size as its derivatives, although they corresponds to $\dot{q}$ and $\ddot{q}$ for now.

### Computing the gravity term
This equation corresponds to the inverse dynamics. We can evaluate parts of it or the entire equation, as we will see next. Let's start with a simple case. 

The gravity term corresponds to the torques when the robot has no velocity nor acceleration:
$$g(q) = b(q,v_q=0) = dyninv(q,v_q=0,a_q=0)$$
In Pinocchio, it can be directly computed with:

In [ ]:
g = pin.computeGeneralizedGravity(rmodel, rdata, q)

![question](question.png)
Define a new cost function that compute the squared norm of the gravity.

In [ ]:
%do_not_load -r 107-117 costs.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li> - The gravity torque can be computed by <code>pin.computeGeneralizedGravity(rmodel, rdata, q)</code>.
     </ul>

</div>

## 7. Weighted gravity cost: RNEA and ABA recursive algorithms

Minimizing the gravity cost is often obtained by bringing the robot to singular configurations. A better behavior might be obtained by a variation of this cost implying the generalized inertia matrix.

### Recursive algorithms in Pinocchio
The 3 most efficient algorithms to evaluate the dynamic equations are implemented in Pinocchio:
- the recursive Newton-Euler algorithm (RNEA) computes the inverse dynamics $\tau_q = invdyn(q,v_q,a_q)$
- the articulated rigid body algorithm (ABA) computes the direct dynamics $a_q = dirdyn(q,v_q,_tau_q)$
- the composite rigid body algorithm (CRBA) computes the mass matrix.
The 3 algorithms can be directly called by their names in Pinocchio.

In [ ]:
vq = np.random.rand(rmodel.nv) * 2 - 1
aq = np.random.rand(rmodel.nv) * 2 - 1
tauq = pin.rnea(rmodel, rdata, q, vq, aq)
aq2 = pin.aba(rmodel, rdata, q, vq, tauq)
assert norm(aq - aq2) < 1e-6
M = pin.crba(rmodel, rdata, q)

The gravity torque $g(q)=b(q,0)$ and the dynamic drift $b(q,vq)$ can be also computed by a truncated RNEA implementation.

In [ ]:
b = pin.nle(rmodel, rdata, q, vq)
assert norm(M @ aq + b - tauq) < 1e-6

### Weighted gravity norm
A better posture regularization can be obtained by taking the norm of the gravity weighted by the inertia:
$$ l(q) = g(q)^T M(q)^{-1} g(q)$$

We can directly evaluate this expression by compute and inverting $M$. Yet it is more efficient to recognize that $M(q)^{-1} g(q) = -dirdyn(q,0,0)$ and can be evaluated by:

In [ ]:
aq0 = -pin.aba(rmodel, rdata, q, np.zeros(rmodel.nv), np.zeros(rmodel.nv))

![question](question.png)
Implement a new cost function for this weighted gravity, and run it as the regularization of a reaching task.

In [ ]:
%do_not_load -r 125-138 costs.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li> Inverse dynamics: <code>tauq = pin.rnea(rmodel, rdata, q, vq, aq)</code>
        <li>Direct dynamcis: <code>aq = pin.aba(rmodel, rdata, q, vq, tauq)</code>
        <li> Inertia matrix: <code>M = pin.crba(rmodel, rdata, q)</code>
        <li> Dynamics drift: <code>b = pin.nle(rmodel, rdata, q, vq)</code>
     </ul>

</div>

## 8. Working with a free floating robot

We now apply the previous mathematical program to a more complex robot with a floatin basis.

### Loading Solo or Talos
You might want to use either the quadruped robot Solo or the humanoid robot Talos for this one.

In [ ]:
# robot = robex.loadTalos()
robot = robex.loadSolo()
viz = Viewer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(loadModel=True)

In [ ]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

In [ ]:
viz.display(robot.q0)
vizutils.addViewerBox(viz, 'world/box', .02, .0324, .0648, [1., .2, .2, .5])
vizutils.addViewerSphere(viz, 'world/ball', .04, [.2, .2, 1., .5])
vizutils.applyViewerConfiguration(viz, 'world/box', [0.5, -.2, .2, 1, 0, 0, 0])
vizutils.applyViewerConfiguration(viz, 'world/ball', [0.5, .2, .2, 1, 0, 0, 0])

In [ ]:
rmodel = robot.model
rdata = rmodel.createData()

### Oh! no, my configuration space is now a Lie group!

In Pinocchio, the floating basis is represented by default as a free-flyer joint. Free-flyer joints have 6 degrees of freedom, but are represented by 7 scalars: the position of the basis center in the world frame, and the orientation of the basis in the world frame stored as a quaternion. This implies that the size of the configuration vector $q$ is one more than the size of the configuration velocity, acceleration or torque: `robot.model.nv`==`robot.model.nv`+1

In [ ]:
print(rmodel.joints[1])
assert rmodel.nq == rmodel.nv + 1

Formally, we say that the configuration $q$ is now living in a Lie group $\mathcal{Q}$, while $v_q$ is in the tangent of this group, the Lie algebra $T_q\mathcal{Q}$. 
In practice, it means that sampling a random $q$ has it was a vector would not work: you must either use the `pin.randomConfiguration` introduced earlier, or normalize the configuration using `pin.normalize`.

In [ ]:
q = np.random.rand(rmodel.nq)  # Not a valid configuration, because the quaternion is not normalized.
q = pin.normalize(rmodel, q)  # Now, it is normalize
# Or better, call directly randomConfiguration
q = pin.randomConfiguration(rmodel)  # q is normalized
viz.display(q)

Similarly, you cannot directly sum a configuration with a velocity: they don't have the same size. Rather, you should integrate the velocity using `pin.integrate`.

In [ ]:
vq = np.random.rand(rmodel.nv) * 2 - 1  # sample a random velocity between -1 and 1.
try:
    q += vq
except:
    print('!!! ERROR')
    print('As expected, this raises an error because q and vq do not have the same dimension.')

q = pin.integrate(rmodel, q, vq)

The reciprocal operation is `pin.difference(rmodel, q1, q2)`: it returns the velocity `vq` so that $q_1 \oplus v_q = q_2$ (with $\oplus$ the integration in $\mathcal{Q}$).

In [ ]:
q1 = pin.randomConfiguration(rmodel)
q2 = pin.randomConfiguration(rmodel)
vq = pin.difference(rmodel, q1, q2)
print(q2 - pin.integrate(rmodel, q1, vq))

Depending on the random samples `q1`,`q2`, the print might be 0, or something different on the quaternion part ... again, because making the vector operation `qa - qb` is not valid in $\mathcal{Q}$.

### Consequence on optimization
So, can we directly use the mathematical program that we defined above with our free-basis robot? Yes and no. 

Let's see that in practice before answering in details.  Just optimize your latest mathematical program with the new robot. If you want the normalization to significantly impact the display, choose a target that is far away from the robot.

**Before runing the next cell**, try to guess how the result will be displayed.
![Think before executing](confused.png)

In [ ]:
mycost = SumOfCost([Cost3d(rmodel, rdata, ptarget = np.array([2, 2, -1]), viz=viz), CostPosture(rmodel, rdata)], [1, 1e-3])
qopt = fmin_bfgs(mycost.calc, robot.q0, callback=mycost.costs[0].callback)

So, what happened?

All the cost functions that we defined are valid when `q` is a proper normalized configuration. **BUT** `fmin_bfgs` is not aware of the normalization constraint. The solver modifies the initial `q` guess into a new `q` vector that is not normalized, hence not a valid configuration.

To prevent that, two solutions are possible:
1. a cost can be added to penalize the solver when choosing a new `q` that is not normalized: `cost(q) = (1 - q[3:7] ** 2) ** 2` (or more generic, `cost(q) = sum((q - pin.normalize(rmodel, q)) ** 2)`.
2. or the vector `q` chosen by the solver should be first normalized when entering `cost.call`, so that the cost is evaluated on a proper configuration.

![Question](question.png)
Redefine your sum-of-cost class with either adding an extra term in the sum to penalize the normalization, or by first normalizing the decision vector `q`.


In [ ]:
%do_not_load -r 2-23 solutions.py

<div class="alert alert-block alert-info">
    <img src="recap.png" title="Recap"/>
    <h3>Recap of the main syntax elements exposed in this section</h3>
    <ul>
        <li><code>pin.integrate</code> adds a configuration with a velocity.</li>
        <li><code>pin.difference</code> makes the difference between two configurations.</li>
        <li><code>pin.normalize</code> project any vector of size NQ into a properly normalized configuration.</li>
     </ul>

</div>